In [1]:
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

%load_ext autoreload
%autoreload 2

In [2]:
from rpy2.robjects import IntVector, Formula

In [3]:
# utils = importr('utils')
# utils.install_packages('ergm')

In [4]:
base = importr('base')
stats = importr('stats')
ergm = importr('ergm')
network = importr('network', on_conflict="warn",
                  robject_translations={
                      'as.tibble.network': 'as_tibble_network',
                      'as_tibble.network': 'as_tibble_network'
                  }
         )

/Users/hangjianli/.pyenv/versions/3.9.2/envs/stergm/lib/python3.9/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
/Users/hangjianli/.pyenv/versions/3.9.2/envs/stergm/lib/python3.9/site-packages/rpy2/robjects/packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "network" to Python symbols: 
-as_tibble_network -> as_tibble.network, as.tibble.network
  warn(msg)


### Load the simulated $A^t$

In [5]:
filename = '../data/edge_mutual__mutual.rds'
readRDS = robjects.r['readRDS']
df = readRDS(filename)
df = np.array(df).astype(int)

## quick tests

In [6]:
yt0=df[0]
yt1=df[1]

In [7]:
p_pos = 2 # edge + mutual
p_neg = 1 # edge
p = p_pos + p_neg

In [257]:
num_nodes = 100
n = 20
T = 50
num_changepts = 5

theta = np.arange(T * p).reshape(T,-1)

In [9]:
form_terms = ['edges', 'mutual']
diss_terms = ['edges']


In [259]:
from mple_learn import stergmGraph
sim1 = stergmGraph(
    X = df,
    form_terms=form_terms,
    diss_terms=diss_terms)

In [218]:
# sim1.gstat(yt0=yt0, yt1=yt1)

In [219]:
lr_form, lr_diss = sim1.g_delta(yt0=yt0, yt1=yt1)
assert lr_form.shape == (n, n, len(form_terms))
assert lr_diss.shape == (n, n, len(diss_terms))

In [261]:
hess = sim1.theta_update_hess(theta, df)

In [262]:
hess.shape

(150, 150)

In [222]:
a = np.arange(5)[::-1]

array([[  0,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8],
       [  9,  10,  11],
       [ 12,  13,  14],
       [ 15,  16,  17],
       [ 18,  19,  20],
       [ 21,  22,  23],
       [ 24,  25,  26],
       [ 27,  28,  29],
       [ 30,  31,  32],
       [ 33,  34,  35],
       [ 36,  37,  38],
       [ 39,  40,  41],
       [ 42,  43,  44],
       [ 45,  46,  47],
       [ 48,  49,  50],
       [ 51,  52,  53],
       [ 54,  55,  56],
       [ 57,  58,  59],
       [ 60,  61,  62],
       [ 63,  64,  65],
       [ 66,  67,  68],
       [ 69,  70,  71],
       [ 72,  73,  74],
       [ 75,  76,  77],
       [ 78,  79,  80],
       [ 81,  82,  83],
       [ 84,  85,  86],
       [ 87,  88,  89],
       [ 90,  91,  92],
       [ 93,  94,  95],
       [ 96,  97,  98],
       [ 99, 100, 101],
       [102, 103, 104],
       [105, 106, 107],
       [108, 109, 110],
       [111, 112, 113],
       [114, 115, 116],
       [117, 118, 119],
       [120, 121, 122],
       [123, 124

In [227]:
a = df.reshape(50, -1)

In [240]:
a = np.arange(24).reshape(4,3,2)
b = np.arange(8).reshape(4,2)

In [235]:
(a - b).squeeze()

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]])

In [243]:
a

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5]],

       [[ 6,  7],
        [ 8,  9],
        [10, 11]],

       [[12, 13],
        [14, 15],
        [16, 17]],

       [[18, 19],
        [20, 21],
        [22, 23]]])

In [244]:
b

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [248]:
 Z = np.sum(a * b[:,np.newaxis, :], axis=2)

In [249]:
Z.shape

(4, 3)

In [250]:
Z

array([[  1,   3,   5],
       [ 33,  43,  53],
       [113, 131, 149],
       [241, 267, 293]])